# Исследование набора данных

Попробуйте исследовать данных из выбранного набора `vega_dataset` с использованием изученных функций:

- различные метки: ``mark_point()``, ``mark_line()``, ``mark_tick()``, ``mark_bar()``, ``mark_area()``, ``mark_rect()``, и т.д.
- различные кодирования: ``x``, ``y``, ``color``, ``shape``, ``size``, ``row``, ``column``, ``text``, ``tooltip``, и т.д.
- группировки и агрегаторы: [Список агрегаторов](https://altair-viz.github.io/user_guide/encoding.html#binning-and-aggregation) можно найти в документации Altair
- панели (``alt.layer`` <-> ``+``, ``alt.hconcat`` <-> ``|``, ``alt.vconcat`` <-> ``&``)

In [109]:
from vega_datasets import data
import altair as alt
import numpy as np
import pandas as pd

In [44]:
iris = data.iris()
iris.head()

,sepalLength,sepalWidth,petalLength,petalWidth,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


*1. Различные метки*

In [5]:
alt.Chart(iris).mark_point().encode(
    color='species',
    y='sepalLength',
    x='sepalWidth'
)

alt.Chart(...)

In [6]:
alt.Chart(iris).mark_point().encode(
    color='species',
    y='petalLength',
    x='petalWidth'
)

alt.Chart(...)

In [7]:
alt.Chart(iris).mark_bar().encode(
    y='species',
    x='mean(petalLength)'
)

alt.Chart(...)

In [8]:
alt.Chart(iris).mark_bar().encode(
    y='species',
    x='count()',
)

alt.Chart(...)

*2. Различные кодирования*

In [48]:
alt.Chart(iris).mark_bar().encode(
    x=alt.X('petalLength', bin=alt.Bin(maxbins=40)),
    y='count()',
    color='species',
    column='species'
)

alt.Chart(...)

In [82]:
alt.Chart(iris).mark_point().encode(
    x='petalLength:Q',
    y='petalWidth:Q',
    color='species:N'
).properties(
    width=300,
    height=300
).facet(
    column='species:N'
)

alt.FacetChart(...)

*3. Группировки и аггрегаторы*

In [91]:
alt.Chart(iris).transform_density(
    "petalLength",
    as_=["petalLength", "density"],
    groupby=["species"],
).mark_area().encode(
    x="petalLength:Q",
    y="density:Q",
).facet(
    "species:N"
)

alt.FacetChart(...)

In [79]:
def violinplot(
    data: pd.DataFrame,
    x: str,
    y: str,
    extent: list = [0, 9],
    shape: tuple=(500, 200),
):
    """Формирование Violinplot."""
    return alt.Chart(data).transform_density(
        y,
        as_=[y, "density"],
        extent=extent,
        groupby=[x],
    ).mark_area(orient="horizontal").encode(
        y=f"{y}:Q",
        color=f"{x}:N",
        x=alt.X(
            "density:Q",
            stack="center",
            impute=None,
            title=None,
            axis=alt.Axis(labels=False, values=[0], grid=False, ticks=True),
        ),
        column=alt.Column(
            f"{x}:N",
            header=alt.Header(
                titleOrient="bottom",
                labelOrient="bottom",
                labelPadding=0,
            ),
        )
    ).properties(
        height=shape[0],
        width=shape[1]
    ).configure_facet(
        spacing=0
    ).configure_view(
        stroke=None
    )


In [84]:
violinplot(data=iris, x="species", y="petalLength")

alt.Chart(...)

*4. Панели*

In [102]:
base = alt.Chart(iris).encode(
    x=alt.X('petalLength', bin=alt.Bin(maxbins=40)),
    y='count()',
    color='species',
)

In [106]:
alt.layer(
    base.mark_line(),
    base.mark_bar(),
)

alt.LayerChart(...)

In [108]:
(base.mark_bar() | base.encode(x=alt.X('petalWidth', bin=alt.Bin(maxbins=40))).mark_bar()) & \
    (base.encode(x=alt.X('sepalLength', bin=alt.Bin(maxbins=40))).mark_bar() | base.encode(x=alt.X('sepalWidth', bin=alt.Bin(maxbins=40))).mark_bar())

alt.VConcatChart(...)

In [126]:
def correlation_matrix(data: pd.DataFrame, shape=(400, 400)):
    """Матрица корреляций."""
    numeric_types = [np.int8, np.int16, np.int32, np.int64, np.float16, np.float32, np.float64]
    numeric_columns = [col for col in data if data[col].dtype in numeric_types]
    data_with_numeric_columns = data[numeric_columns]

    corr_data = \
        data_with_numeric_columns.corr().stack().reset_index().rename(
        columns={
            0: "correlation",
            "level_0": "variable_1",
            "level_1": "variable_2",
        })
    corr_data["visible_correlation"] = corr_data["correlation"].map("{:.3f}".format)

    plot_base = alt.Chart(corr_data).encode(
        x="variable_1:O",
        y="variable_2:O",   
    )

    text = plot_base.mark_text().encode(
        text="visible_correlation",
        color=alt.condition(
            alt.datum.correlation > 0.5, 
            alt.value("white"),
            alt.value("black"),
        )
    )

    corr_plot = plot_base.mark_rect().encode(
        color="correlation:Q"
    )

    return (corr_plot + text).properties(height=shape[0], width=shape[1])

In [127]:
correlation_matrix(data=iris)

alt.LayerChart(...)